# pyntcloud QuickStart 

The following notebook is a **real-world** example of using [pyntcloud](https://github.com/daavoo/pyntcloud).

---

It's our first day at the Jurassic Park Computer Vision Lab!!

We are given a 3D scene simulating the point clouds that are generated from a
lidar sensor on top of one of the Jurassic Park self-driving cars.

Here is a view of the 3D scene (I'm not very skillful at 3D modelling):

![view of scene](data/1.png)

### In order to run this tutorial locally, download the scene from [this link](https://mega.nz/#!qIZ13T7B!USyVexfYb8J3SDC_3h3HVUhFMXE2z0qUZNn3Ov4GXX0).

We will be only working with the parts of the scene that are visible from
the front of the car, in order to simulate what a real lidar sees.

**Our task** is to help the JP car chosing whether it should turn to the left, where
there is an apacible ankylosaurus, or right, where there is a hungry tyranosaurus.


---

Most of pyntcloud's functionallity can be accesed through the core class: [PyntCloud](http://pyntcloud.readthedocs.io/en/latest/PyntCloud.html).

In [1]:
from pyntcloud import PyntCloud

And this is the only import we need for now.

## Loading 3D data

The first thing we are going to do is load the 3D data into Python, using PyntCloud's class method: [from_file](http://pyntcloud.readthedocs.io/en/latest/io.html)

In [2]:
scene = PyntCloud.from_file("data/visible.ply")

We can get some basic information of the scene using the internal `__repr__` method:

In [3]:
scene

PyntCloud
51777 points with 7 scalar fields
99059 faces in mesh
0 kdtrees
0 voxelgrids
Centroid: -17.429340362548828, 7.901965618133545, 5.268535137176514
Other attributes:

This allows us to realize that the file is actually a triangular mesh. We can tell this because there are 99059 faces in mesh.

In [4]:
scene.mesh.head()

,v1,v2,v3
0,0,2,1
1,0,3,2
2,37,83,36
3,37,38,84
4,39,84,38


Those lazy bastards from the desing deparment didn't convert the 3D model into a point cloud, wich is the real output from a lidar scanner.

## 3D point cloud from 3D mesh

Luckely for us, we can convert a triangular mesh into a point cloud:

In [5]:
# sample 500000 points from the mesh
sampled_points = scene.get_sample("mesh_random_sampling",
                                  n=500000,
                                  rgb=True,
                                  normals=True)

# manually construcst a new PyntCloud with those points
scene = PyntCloud(sampled_points)

We can verify the result using the `__repr__` method mentioned above (this time using the `print` function) ...

In [6]:
print(scene)

PyntCloud
500000 points with 6 scalar fields
0 faces in mesh
0 kdtrees
0 voxelgrids
Centroid: -10.081748008728027, 0.9480186104774475, 0.3907921612262726
Other attributes:



... Take a look at the information in points ...

In [7]:
scene.points.head()

,x,y,z,red,green,blue,nx,ny,nz
0,-10.218865,0.106860,0.000000,0,28,0,0.000000,0.000000,1.000000
1,-17.654833,-19.405439,0.000000,0,85,0,0.000000,0.000000,1.000000
2,-15.657837,24.334534,7.097624,105,66,44,0.907271,0.078978,0.413065
3,-18.207640,0.631519,0.000000,0,28,0,0.000000,0.000000,1.000000
4,-12.181130,-39.528580,0.000000,0,85,0,0.000000,0.000000,1.000000


... And visualize the point cloud right inside the notebook!

In [8]:
scene.plot()

## Floor segmentation

We are not interested in the points that compose the floor, so we will use [RANSAC](https://es.wikipedia.org/wiki/RANSAC) to find the plane that contains those points.

This is a good example of using the function [add_scalar_field](http://pyntcloud.readthedocs.io/en/latest/scalar_fields.html), wich will add a new column to `points` and return a `string` with the name of that column.

It's a good practice to store the strings returned `from PyntCloud.`**add_**`*` methods. As you will see later, this it quite convenient when chaining operations that require those strings as arguments.

In [9]:
is_floor = scene.add_scalar_field("plane_fit",
                                  n_inliers_to_stop=len(scene.points)/30)

This method adds a new column to points.

In [10]:
scene.points.head()

,x,y,z,red,green,blue,nx,ny,nz,is_plane
0,-10.218865,0.106860,0.000000,0,28,0,0.000000,0.000000,1.000000,1
1,-17.654833,-19.405439,0.000000,0,85,0,0.000000,0.000000,1.000000,1
2,-15.657837,24.334534,7.097624,105,66,44,0.907271,0.078978,0.413065,0
3,-18.207640,0.631519,0.000000,0,28,0,0.000000,0.000000,1.000000,1
4,-12.181130,-39.528580,0.000000,0,85,0,0.000000,0.000000,1.000000,1


In this case, the name of the new scalar field ('is_plane') is as large as the name of the variable where we store it.

However you will find out that many scalar field have a more complex names, and thus is more convenient to always store them in a variable.

We can now visualize the point cloud colored with the new scalar field.

The `plot` function allows us to define some usefull keyword arguments.

In [11]:
help(PyntCloud.plot)

Help on function plot in module pyntcloud.core_class:

plot(self, use_as_color=['red', 'green', 'blue'], cmap='hsv', output_name='pyntcloud_plot', width=800, height=500)
    Visualize PyntCloud in a Jupyter notebook using three.js.
    
    Parameters
    ----------
    use_as_color: str or ["red", "green", "blue"], optional
        Default: ["red", "green", "blue"]
        Indicates wich scalar fields will be used to colorize the rendered
        point cloud.
    
    cmap: str, optional
        Default: "hsv"
        Color map that will be used to convert a single scalar field into rgb.
        Check matplotlib cmaps.
    
    output_name: str, optional
        Default: "pyntcloud_plot"
        Base filename that will be used to create:
            output_name.html
            output_name.ply
            output_name.json
    
    width: int, optional
        Default: 800
        Adjusts the size of the IFrame plotted in Jupyter notebook.
    
    height: int, optional
        Default

In [12]:
scene.plot(use_as_color=is_floor, cmap="RdYlGn", output_name="is_floor")

Now we can use the added scalar field to extract the non-floor parts of the scene.

In [13]:
# creates a boolean array
not_floor = scene.points[is_floor] != 1 

In [14]:
scene.apply_filter(not_floor)

In [15]:
scene.points.head()

,x,y,z,red,green,blue,nx,ny,nz,is_plane
0,-15.657837,24.334534,7.097624,105,66,44,0.907271,0.078978,0.413065,0
1,-16.364546,-21.531874,1.977043,39,39,34,0.449040,-0.618624,-0.644722,0
2,-16.666334,25.210224,1.399786,49,30,23,0.694688,-0.713848,-0.088485,0
3,-15.166573,26.397392,2.580655,118,83,60,0.930862,0.016087,0.365018,0
4,-17.368053,21.308783,9.124968,123,116,76,0.650959,-0.405334,0.641839,0


In [16]:
scene.plot(output_name="without_floor")

## Clustering dinosaurs

Let's separate the points that belong to each dinosaur using a simple clustering technique called euclidean clustering. 

This is a good example of chaining `add_` operations storing the string indetifiers in variables.

First, we will add a VoxelGrid to the PyntCloud using [add_structure](http://pyntcloud.readthedocs.io/en/latest/structures.html).

The `sizes` parameter is quite important. If the distance between one query point and it's closest point in some cluster is higher than `size`, the point won't be considered part of that cluster.

In this case, we now that Dinosaurs are pretty damm big, and they are not really close, so let's set a separation of 3 metters along the 3 axis.

In [17]:
vg_id = scene.add_structure("voxelgrid", sizes=[3,3,3])

Using the voxelgrid, we can add a new scalar field that indicates to wich cluster each point belong.

In [18]:
clusters_id = scene.add_scalar_field("euclidean_clusters", voxelgrid=vg_id)

And visualize the scene colored according to those clusters:

In [19]:
scene.plot(use_as_color=clusters_id, cmap="cool")

## Using Deep Learning to clasify dinosaurs

Deep learning is quite cool, so why don't we use it to classify wich of the clusters
is an herbibore dinosaur and wich one a carnivore.

Deep learning with 3D data involves some complications compared with 2D (images).

Luckely for us, pyntcloud includes the `learn` module wich make things a lot easier.

First of all, let's use the cluster information to split the point cloud, create
a PyntCloud for each dinosaur, and save each one:

In [20]:
scene.split_on(clusters_id, save_path="tmp")

## TODO

In order to make use of deep learning, normally we have to desing a model and train
the model with tons of data before be able to make any good predictions.

Luckely for us in pyntcloud.learn there are some pre-defined and pre-trained models
ready to be finetuned for our own pourpuses.

There is even a model already trained that can classify dinosaurs into herbibores
and carnivores, how convenient!

To make use of all this functionallity we need to first import the learn module:

## Clean directory

In [21]:
import os
from shutil import rmtree
from utils import clean_visualizations

In [22]:
clean_visualizations(os.getcwd())
rmtree("tmp")

Removing pyntcloud_plot.html
Removing is_floor.html
Removing pyntcloud_plot.ply
Removing without_floor.html
Removing without_floor.ply
Removing is_floor.ply
